In [125]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
import gym
import gym_combrf
import matplotlib.pyplot as plt

import numpy as np

In [128]:
from Source.misc_fun.utils import Generate_Beams, var_plotbeam

beamset = Generate_Beams(8,np.array([np.pi/8, np.pi/4]))

print(beamset)

[(0.39269908, 8) (0.78539816, 8) (1.17809725, 8) (1.57079633, 8)
 (1.96349541, 8) (2.35619449, 8) (2.74889357, 8) (3.14159265, 8)
 (0.78539816, 4) (1.57079633, 4) (2.35619449, 4) (3.14159265, 4)]


In [127]:
#create a channel variation of vectors, one for each location in environment

#create same channel variation over time for all location, as we train MAB separately for every location.
#This can then be compared with DQN with same channel variation over time.

inp_fptr = open("ch_variation_large.txt")
ch_randvals = inp_fptr.read().splitlines()
ch_randvals = [np.complex(a.replace('i','j')) for a in ch_randvals]


In [182]:
env = gym.make('combrf-v7')
env.ant_arr = 'ula'
env.ch_model = 'uma-los'
env.init_ch_model = 'uma-los'
env.sc_xyz = np.array([[0,20,21.5], [20,0,21.5],[-20,0,21.5],[0,-20,21.5]])
rx_xcov = np.array([20])
rx_ycov = np.array([20])
rx_zcov = np.array([41.5])
beam_param = 'beam-narrow'
beam_paramvals = np.array([])
env.set_beamparameters(beam_param, beam_paramvals)
env.set_coverage(rx_xcov,rx_ycov,rx_zcov)
obs = env.reset(0,ch_randvals[0], 0.5)
#print(obs)
#print(obs.shape)
print(env.ch_model)
print(env.tx_loc)
print(env.tx_locs)
#print(env.obs_space.sample())
#print(env.rx_xcov)

loc_ndx = 0
#for txdir_ndx in range(env.N_tx):
obs = env.reset(loc_ndx,ch_randvals[0], 0.5)
#min_rate, max_rate, min_exh_ndx, max_exh_ndx, _, _ = env.get_minmax_exhrate(loc_ndx, ch_randvals[0])
#print("max_rate: {}, exh action ndx: {}".format(max_rate, max_exh_ndx))

#print("init_obs: ", obs)
max_rate = 0.0
best_rxdir_ndx = -1
best_rxdir_ndx = -1

for txdir_ndx in range(env.N_tx):
    for rxdir_ndx in range(env.N_rx):
        obs,_,_,_ = env.step(txdir_ndx*env.N_rx+rxdir_ndx)
        #print("obs: ", obs)
        rate = env.rate
        if (rate > max_rate):
            max_rate = rate
            best_rxdir_ndx = rxdir_ndx
            best_txdir_ndx = txdir_ndx
        #if (txdir_ndx == 46) or (txdir_ndx ==45):
        print("txdir_ndx: {}, rxdir_ndx: {}, rate: {},rssival:({},{})".format(txdir_ndx, rxdir_ndx, rate,env.rssi_val.real, env.rssi_val.imag))
    print("\n")
print("best_txdir_ndx: {}, best_rxdir_ndx:{}, best_rate:{}\n".format(best_txdir_ndx, best_rxdir_ndx, max_rate))
#print("txdir_ndx: {}, best rxdir_ndx: {}, best_rate: {}, min_rate:{}".format(txdir_ndx, max_rxdir_ndx, max_rate, min_rate))

init rate: -inf, init tx action: -1, init rx action: -1
uma-av-nlos
[[20.  20.  41.5]]
[array([[20. , 20. , 41.5]])]
init rate: -inf, init tx action: -1, init rx action: -1
txdir_ndx: 0, rxdir_ndx: 0, rate: 14.336278666383517,rssival:(2.1006490064157652e-06,-7.265528463828244e-07)
txdir_ndx: 0, rxdir_ndx: 1, rate: 19.710334083858893,rssival:(1.0425139921386234e-05,9.80925299767363e-06)
txdir_ndx: 0, rxdir_ndx: 2, rate: 13.371808583200831,rssival:(-1.2550539184337954e-06,9.78068738009216e-07)
txdir_ndx: 0, rxdir_ndx: 3, rate: 20.5149935481115,rssival:(-1.7420802348010567e-05,7.377683692531432e-06)
txdir_ndx: 0, rxdir_ndx: 4, rate: 16.601412453069205,rssival:(-3.6986926773573177e-06,-3.1733297628535372e-06)
txdir_ndx: 0, rxdir_ndx: 5, rate: 12.69510536439437,rssival:(6.368002188504585e-07,-1.0854767136879014e-06)
txdir_ndx: 0, rxdir_ndx: 6, rate: 17.322270020886585,rssival:(-5.569976616377472e-06,-2.8495482693463578e-06)
txdir_ndx: 0, rxdir_ndx: 7, rate: 16.101627244149608,rssival:(3.968

In [178]:
from Source.misc_fun import channel_mmW

loc_ndx = 0
#for txdir_ndx in range(env.N_tx):
#obs = env.reset(loc_ndx,ch_randvals[0], 0.5)

env = gym.make('combrf-v7')
env.ant_arr = 'ula'
env.ch_model = 'uma-los'
env.init_ch_model = 'uma-los'
#env.sc_xyz = np.array([[0,-20,21.5], [-20,0,21.5],[20,0,21.5],[0,20,21.5]])
env.sc_xyz = np.array([[0,20,21.5], [20,0,21.5],[-20,0,21.5],[0,-20,21.5]])
rx_xcov = np.array([60])
rx_ycov = np.array([20])
rx_zcov = np.array([41.5])
beam_param = 'beam-narrow'
beam_paramvals = np.array([])
env.set_beamparameters(beam_param, beam_paramvals)
env.set_coverage(rx_xcov,rx_ycov,rx_zcov)
obs = env.reset(0,ch_randvals[0], 0.5)

channel = channel_mmW.Channel(env.freq, env.tx_locs[loc_ndx], env.rx_loc, env.sc_xyz, 'model', env.init_ch_model, 'nrx', env.N_rx,
                                   'ntx', env.N_tx, 'nFFT', env.nFFT, 'df', env.df)

ch_randval=None
channel.generate_paths(ch_randval,loc_ndx)
#h = channel.get_h()
#print(h)

print("ch coeff0: ",np.abs(channel.coeff[0]))
print("ch coeff1: ",np.abs(channel.coeff[1]))
print("ch coeff2: ",np.abs(channel.coeff[2]))
print("ch coeff3: ",np.abs(channel.coeff[3]))
print("ch coeff4: ",np.abs(channel.coeff[4]))

print("path loss0: ", channel.pathloss[0])
print("path loss1: ", channel.pathloss[1])
print("path loss2: ", channel.pathloss[2])
print("path loss3: ", channel.pathloss[3])
print("path loss4: ", channel.pathloss[4])
#min_rate, max_rate, min_exh_ndx, max_exh_ndx, _, _ = env.get_minmax_exhrate(loc_ndx)
#print("max_rate: {}, exh action ndx: {}".format(max_rate, max_exh_ndx))


init rate: -inf, init tx action: -1, init rx action: -1
ch coeff0:  1.3366496319292097e-05
ch coeff1:  2.7781039029809163e-06
ch coeff2:  3.842465534697695e-06
ch coeff3:  1.8645114935032962e-06
ch coeff4:  2.2179834186419684e-06
path loss0:  97.47964833734551
path loss1:  111.12503030769585
path loss2:  108.30780038499759
path loss3:  114.58869870301595
path loss4:  113.08083409808535


In [156]:
env = gym.make('combrf-v7')
env.ant_arr = 'ula'
env.ch_model = 'uma-los'
env.init_ch_model = 'uma-los'
env.sc_xyz = np.array([])
rx_xcov = np.array([60])
rx_ycov = np.array([20])
rx_zcov = np.array([41.5])
beam_param = 'beam-narrow'
beam_paramvals = np.array([])
env.set_beamparameters(beam_param, beam_paramvals)
env.set_coverage(rx_xcov,rx_ycov,rx_zcov)
obs = env.reset(0,ch_randvals[0], 0.5)
#print(obs)
#print(obs.shape)
print(env.ch_model)
print(env.tx_loc)
#print(env.obs_space.sample())
#print(env.rx_xcov)

loc_ndx = 0
#for txdir_ndx in range(env.N_tx):
#min_rate, max_rate, min_rxdir_ndx, max_rxdir_ndx, _, _ = env.get_minmax_exhrate(loc_ndx, txdir_ndx, ch_randvals[0])
obs = env.reset(loc_ndx,ch_randvals[0], 0.5)
#print("init_obs: ", obs)
max_rate = 0.0
best_rxdir_ndx = -1
best_rxdir_ndx = -1

for txdir_ndx in range(env.N_tx):
    for rxdir_ndx in range(env.N_rx):
        obs,_,_,_ = env.step(txdir_ndx*env.N_rx+rxdir_ndx)
        #print("obs: ", obs)
        rate = env.rate
        if (rate > max_rate):
            max_rate = rate
            best_rxdir_ndx = rxdir_ndx
            best_txdir_ndx = txdir_ndx
        #if (txdir_ndx == 46) or (txdir_ndx ==45):
        print("txdir_ndx: {}, rxdir_ndx: {}, rate: {},rssival:({},{})".format(txdir_ndx, rxdir_ndx, rate,env.rssi_val.real, env.rssi_val.imag))
    print("\n")
print("best_txdir_ndx: {}, best_rxdir_ndx:{}, best_rate:{}\n".format(best_txdir_ndx, best_rxdir_ndx, max_rate))
#print("txdir_ndx: {}, best rxdir_ndx: {}, best_rate: {}, min_rate:{}".format(txdir_ndx, max_rxdir_ndx, max_rate, min_rate))

init rate: -inf, init tx action: -1, init rx action: -1
uma-av-los
[[60.  20.  41.5]]
init rate: -inf, init tx action: -1, init rx action: -1
txdir_ndx: 0, rxdir_ndx: 0, rate: 23.29964693138604,rssival:(-1.9106675430730673e-05,4.5839100587568486e-05)
txdir_ndx: 0, rxdir_ndx: 1, rate: 18.37948659978521,rssival:(-3.2058127177700226e-06,-8.43679162373573e-06)
txdir_ndx: 0, rxdir_ndx: 2, rate: 15.708401264591538,rssival:(-2.2336252970339074e-07,3.569215750399157e-06)
txdir_ndx: 0, rxdir_ndx: 3, rate: 16.883721526963946,rssival:(-4.857725474556862e-06,2.2991954548020222e-06)
txdir_ndx: 0, rxdir_ndx: 4, rate: 16.13523180936815,rssival:(3.3645685825931095e-06,2.42321648720871e-06)
txdir_ndx: 0, rxdir_ndx: 5, rate: 18.998628710405484,rssival:(5.567561132174798e-06,9.701377879435358e-06)
txdir_ndx: 0, rxdir_ndx: 6, rate: 21.091917213496643,rssival:(2.2129672006552727e-05,6.645781252367959e-06)
txdir_ndx: 0, rxdir_ndx: 7, rate: 22.77967734729146,rssival:(1.7742198562961128e-05,3.748560382456196e

In [ ]:
#Testing the max possible reward
from Source.antenna import ula
from Source.misc_fun.geometry import cart2sph
from Source.misc_fun.conversion import *

def compute_rate(r, theta, txbdir_ndx, wRF):
    tx_loc = np.array([[r*np.cos(theta), r*np.sin(theta), 22.5]])
    sc_xyz = np.array([])

    obs = env.test_reset(tx_loc, txbdir_ndx, sc_xyz, np.exp(1j * 2 * np.pi * 0.6))
    #print(env.h)
    tx = env.tx_loc
    rx = env.rx_loc
    (az_aoa, el_aoa, temp) = cart2sph(tx[0, 0] - rx[0, 0], tx[0, 1] -rx[0, 1], tx[0, 2] - rx[0, 2])
    
    #self.tx_beam = ula.var_steervec(self.N_tx, self.tx_bdir, 0)
    #self.eff_ch = np.array(self.h[:, :, 0]).dot(self.tx_beam)
    #print("eff_ch: ", env.eff_ch)
    rssi_val = np.sqrt(env.N_rx * env.N_tx) * np.array(np.conj(wRF.T).dot(env.eff_ch)) #+ (np.conj(wRF.T).dot(env.noise))[0]
    
    #next_obs, rwd, done, _ = env.step(rxbdir_ndx)
    #print(rssi_val)
    #print((np.conj(wRF.T).dot(env.noise))[0])
    Es = db2lin(env.P_tx) #* (1e-3 / self.B)
    SNR = Es * np.abs(rssi_val)**2 / (env.N0*env.B)
    rate = np.log2(1 + SNR)  # in Gbit/s (self.B / self.nFFT) *
    return rssi_val, rate
#As the max reward is close to 1, calculation of reward based on reward seems to be good, 
#especially with random tx_beam and additional noise

In [ ]:
#best narrow beam
r1 = 1
theta1 = 0*np.pi/180
txbdir_ndx1 = 0
rxbdir_ndx1 = 7
_,rate1 = compute_rate(r1, theta1,txbdir_ndx1, rxbdir_ndx1)
print("txloc: ", np.array([[r1*np.cos(theta1), r1*np.sin(theta1), 0]]))
print("narrow beam rate1: ", rate1)
print("rxbeam: ",(env.BeamSet[rxbdir_ndx1][0]*180/np.pi,env.BeamSet[rxbdir_ndx1][1]))
print("codeword: ", ula.var_steervec(env.N_rx, env.BeamSet[rxbdir_ndx1] , 0))
#best wide beam
r2 = 1
theta2 = 0*np.pi/180
txbdir_ndx2 = 0
rxbdir_ndx2 = 11
_,rate2 = compute_rate(r2, theta2,txbdir_ndx2, rxbdir_ndx2)
print("txloc: ", np.array([[r2*np.cos(theta2), r2*np.sin(theta2), 0]]))
print("wide beam rate2: ", rate2)
print("rxbeam: ",(env.BeamSet[rxbdir_ndx2][0]*180/np.pi,env.BeamSet[rxbdir_ndx2][1]))
print("codeword: ", ula.var_steervec(env.N_rx, env.BeamSet[rxbdir_ndx2] , 0))

theta_1, gr_1 = var_plotbeam(env.BeamSet[rxbdir_ndx1], 8)
theta_2, gr_2 = var_plotbeam(env.BeamSet[rxbdir_ndx2],4)
ax = plt.subplot(111, projection='polar')
##print(theta.shape, gr.shape)
ax.plot(theta_1, gr_1, theta_2, gr_2)
#plt.pause(0.001)
ax.scatter(theta1, r1, c='r')
plt.show()

In [ ]:
#worst narrow beam
r1 = 100
theta1 = 45*np.pi/180
txbdir_ndx1 = 0
rxbdir_ndx1 = 1
wRF = ula.var_steervec(env.N_rx, (env.BeamSet[rxbdir_ndx1][0]-0*np.pi/180,env.BeamSet[rxbdir_ndx1][1]), 0)
_,rate1 = compute_rate(r1, theta1,txbdir_ndx1, wRF)
print("txloc: ", np.array([[r1*np.cos(theta1), r1*np.sin(theta1), 0]]))
print("narrow beam rate1: ", rate1)
print("rxbeam: ",((env.BeamSet[rxbdir_ndx1][0]-0*np.pi/180)*180/np.pi,env.BeamSet[rxbdir_ndx1][1]))
print("codeword: ", ula.var_steervec(env.N_rx, env.BeamSet[rxbdir_ndx1] , 0))

#best wide beam
r2 = 100
theta2 = 45*np.pi/180
txbdir_ndx2 = 0
rxbdir_ndx2 = 8
wRF = ula.var_steervec(env.N_rx, env.BeamSet[rxbdir_ndx2], 0)
_,rate2 = compute_rate(r2, theta2,txbdir_ndx2, wRF)
print("txloc: ", np.array([[r2*np.cos(theta2), r2*np.sin(theta2), 0]]))
print("wide beam rate2: ", rate2)
print("rxbeam: ",(env.BeamSet[rxbdir_ndx2][0]*180/np.pi,env.BeamSet[rxbdir_ndx2][1]))
print("codeword: ", ula.var_steervec(env.N_rx, env.BeamSet[rxbdir_ndx2] , 0))

theta_1, gr_1 = var_plotbeam(env.BeamSet[rxbdir_ndx1], 8)
theta_2, gr_2 = var_plotbeam(env.BeamSet[rxbdir_ndx2],4)
ax = plt.subplot(111, projection='polar')
##print(theta.shape, gr.shape)
ax.plot(theta_1, gr_1, theta_2, gr_2)
#plt.pause(0.001)
#ax.scatter(theta1, r1, c='r')
plt.show()

In [ ]:
from Source.misc_fun.utils import Generate_BeamDirs, plotbeam
import numpy as np
r=1
beamset = Generate_BeamDirs(8,np.array([0]))
print(beamset)

ax = plt.subplot(111, projection='polar')
for i in range(len(beamset)):
    
    theta, gr = plotbeam(beamset[i],8)
    ax.plot(theta, gr, color='blue')

theta, gr = plotbeam(beamset[1]+11.25*np.pi/180,8)
ax.plot(theta, gr, color='green')

ax.scatter(beamset[1]+11.25*np.pi/180, r, c='r')
#plt.savefig("ploar_withrefinement.png")
plt.show()

In [ ]:
#worst narrow beam
r1 = 1
theta1 = 67.5*np.pi/180
txbdir_ndx1 = 0
rxbdir_ndx1 = 3
_,rate1 = compute_rate(r1, theta1,txbdir_ndx1, rxbdir_ndx1)
print("txloc: ", np.array([[r1*np.cos(theta1), r1*np.sin(theta1), 21.5]]))
print("narrow beam rate1: ", rate1)
print("rxbeam: ",(env.BeamSet[rxbdir_ndx1][0]*180/np.pi,env.BeamSet[rxbdir_ndx1][1]))
print("codeword: ", ula.var_steervec(env.N_rx, env.BeamSet[rxbdir_ndx1] , 0))

#worst wide beam
r2 = 1
theta2 = 67.5*np.pi/180
txbdir_ndx2 = 0
rxbdir_ndx2 = 9
_,rate2 = compute_rate(r2, theta2,txbdir_ndx2, rxbdir_ndx2)
print("txloc: ", np.array([[r2*np.cos(theta2), r2*np.sin(theta2), 0]]))
print("wide beam rate2: ", rate2)
print("rxbeam: ",(env.BeamSet[rxbdir_ndx2][0]*180/np.pi,env.BeamSet[rxbdir_ndx2][1]))
print("codeword: ", ula.var_steervec(env.N_rx, env.BeamSet[rxbdir_ndx2] , 0))

theta_1, gr_1 = var_plotbeam(env.BeamSet[rxbdir_ndx1], 8)
theta_2, gr_2 = var_plotbeam(env.BeamSet[rxbdir_ndx2],4)
ax = plt.subplot(111, projection='polar')
##print(theta.shape, gr.shape)
ax.plot(theta_1, gr_1, theta_2, gr_2)
#plt.pause(0.001)
ax.scatter(theta1, r1, c='r')
plt.show()

In [ ]:
r2 = 100
theta2 = 45*np.pi/180
txbdir_ndx2 = 0

#rxbdir_ndx2 = 9
rss_vals=[]
rate_vals=[]
widerss_vals=[]
widerate_vals =[]
for rxbdir_ndx in range(len(env.BeamSet)):
    print(env.BeamSet[rxbdir_ndx])
    wRF = ula.var_steervec(env.N_rx, env.BeamSet[rxbdir_ndx], 0)
    rssi_val,rate = compute_rate(r2, theta2,txbdir_ndx2, wRF)
    #print(rssi_val)
    if(rxbdir_ndx <=7):
    #    if (rxbdir_ndx == 1):
    #        print(wRF)
    #        print([x*2.78/y for x,y in zip(wRF, env.eff_ch)])
        rss_vals.append(np.abs(rssi_val)**2)
        rate_vals.append(rate)
    else:
    #    if (rxbdir_ndx == 8):
    #        print(env.BeamSet[rxbdir_ndx])
    #        print(rate)
    #        print(np.abs(rssi_val)**2)
    #        print(wRF)
    #        print([x/y for x,y in zip(wRF, env.eff_ch)])
        widerss_vals.append(np.abs(rssi_val)**2)
        widerate_vals.append(rate)
        
fig = plt.figure(figsize=(9,6))
plt.plot(np.arange(len(rss_vals)), rss_vals, np.arange(0,len(rss_vals),2), widerss_vals)
plt.legend(["narrow beamwidth", "wider beamwidth"])
plt.title("r: {}, theta: {}, Ntx: {}, Nrx: {}".format(r2, theta2*180/np.pi, env.N_tx, env.N_rx))
plt.show()

In [ ]:
fig = plt.figure(figsize=(9,6))
plt.plot(np.arange(len(rate_vals)), rate_vals, np.arange(0,len(rate_vals),2), widerate_vals)
print("best narrow beam rate: {}, best wider beam rate: {}".format(max(rate_vals), max(widerate_vals)))
plt.legend(["narrow beamwidth", "wider beamwidth"])
plt.xticks(np.arange(0, len(rate_vals)), [str(np.round(x[0]*180/np.pi, decimals=2)) for x in env.BeamSet[0:len(rate_vals)]])
plt.xlabel('RX beam angle (deg)')
plt.ylabel('data rate (bits/s)')
plt.title("r: {}, theta: {}, Ntx: {}, Nrx: {}".format(r2, theta2*180/np.pi, env.N_tx, env.N_rx))
#plt.savefig("140820_wide_narrow_rate_2.pdf")
plt.show()

In [ ]:
obs = env.reset()
#print(obs)
print("cap: ", env.get_capacity())
print("TX loc: ", env.tx_loc)
i=0
while True:
    action = env.action_space.sample()
    obs, rwd, done, _ = env.step(action)
    #if (i==0):
    #    print(obs)
    #    i+=1
    print(action, rwd, env.rate)
    
    if done:
        break

In [ ]:
sc = np.array([[0,100,0], [10,50,0], [40,60,0], [70,80,0]])
obs = env.test_reset(np.array([[100,0,0]]), 2, sc)
print(env.tx_loc)
print(env.tx_bdir*(180/np.pi))
print(env.rx_bdir*(180/np.pi))
rate, ndx, rssi_val = env.get_exh_rate()
print(rssi_val)
print(rate)
print(env.channel.pathloss)

In [ ]:
tx_loc = np.array([[100,0,0]])
sc = np.array([[0,100,0]])

best_rssi_val = 0
best_rate = 0.0
best_rx_ndx = 0
best_tx_ndx = 0
for tx_ndx in range(env.N_tx):
    obs = env.test_reset(tx_loc, tx_ndx, sc)
    rate, rx_ndx, rssi_val = env.get_exh_rate()
    if (rate > best_rate):
        best_rate = rate
        best_rssi_val = rssi_val
        best_rx_ndx = rx_ndx
        best_tx_ndx = tx_ndx    

print(tx_loc)
print("TX bdir: ", env.BeamSet[best_tx_ndx]*(180/np.pi))
print("RX bdir: ", env.BeamSet[best_rx_ndx]*(180/np.pi))
print(best_rssi_val)
        

In [ ]:
#Plot the curves for RSSI vs rx bdir for two different tx_directions

tx_loc = np.array([[100,0,0]])
sc = np.array([[0,100,0], [10,50,0], [40,60,0], [70,80,0]])

obs_1 = env.test_reset(tx_loc, 0, sc)
rssi_values1, rate_values1 = env.meas_rate()
rssi_values1 = [np.abs(x) ** 2 for x in rssi_values1]
yaxis1 = list(reversed(np.sort(rssi_values1)))

obs_2 = env.test_reset(tx_loc, 7, sc)
rssi_values2, rate_values2 = env.meas_rate()
rssi_values2 = [np.abs(x) ** 2 for x in rssi_values2]
print(rssi_values2)
yaxis2 = list(reversed(np.sort(rssi_values2)))

fig = plt.figure()
#ax1 = fig.add_subplot(131)
xaxis1 = []
for val in yaxis1:
    ndx = rssi_values1.index(val)
    xaxis1.append(ndx)
    
xaxis2 = []
for val in yaxis2:
    ndx = rssi_values2.index(val)
    xaxis2.append(ndx)

print(yaxis1)
print(xaxis1)

#plt.plot(xaxis1, yaxis1,xaxis2, yaxis2 )
plt.plot(np.arange(1, env.N_tx+1), rssi_values1,np.arange(1, env.N_tx+1), rssi_values2 )

plt.ylabel('RSSI')
plt.xlabel('RXdir')
plt.legend(['TX dir0', 'TXdir1'])
plt.show()
